In [0]:
from pyspark.sql.functions import expr, regexp_replace, trim, col

# Read both datasets
df_sdoh = spark.table("civAI.raw.sdoh_age_population")
df_dist = spark.table("civAI.raw.age_group_distribution")

# Unpivot age columns from df_dist
age_columns = [col for col in df_dist.columns if col.startswith("total_") and col != "total_total"]
expr_str = ",".join([f"'{col}', `{col}`" for col in age_columns])

df_dist_long = df_dist.selectExpr(
    "Number as region",
    f"stack({len(age_columns)}, {expr_str}) as (age_group_col, population)"
)

# Clean age group column
df_dist_long = df_dist_long.withColumn(
    "age_group",
    trim(regexp_replace("age_group_col", "total_", ""))
).drop("age_group_col")

# Add year
df_dist_long = df_dist_long.withColumn("year", expr("2024"))

# Ensure population is treated as string in both sources before union
df_sdoh = df_sdoh.select(
    col("age_group"),
    col("year"),
    col("population").cast("string").alias("population")
)

df_dist_long = df_dist_long.select(
    col("age_group"),
    col("year"),
    col("population").cast("string").alias("population")
)

# Union
df_combined = df_sdoh.unionByName(df_dist_long)

# Filter numeric-only population rows
filtered_df = df_combined.filter(col("population").rlike("^[0-9]+(\\.[0-9]+)?$"))

# Safe cast to double
cleaned_df = filtered_df.withColumn("population", col("population").cast("double"))

# Write to Delta
cleaned_df.write.mode("overwrite").format("delta").saveAsTable("civAI.raw.combined_age_population")
